## 7.3.2 Hyperas를 이용한 하이퍼파라미터 최적화 구현하기

- https://github.com/maxpumperla/hyperas

In [11]:
# !pip install hyperas

import numpy as np

from hyperopt import Trials, STATUS_OK, tpe
from keras.datasets import mnist
from keras import layers, models

from hyperas import optim
from hyperas.distributions import choice, uniform

In [16]:
# create data() function

def data():
    (train_x, train_y), (test_x, test_y) = mnist.load_data()
    train_x = train_x.astype('float32').reshape([60000, -1]) / 255.
    test_x = test_x.astype('float32').reshape([10000, -1]) / 255. 
    
    return train_x, train_y, test_x, test_y


# create model function
def mnist_model(train_x, train_y, test_x, test_y):
    model = models.Sequential()
    model.add(layers.Dense(256, activation='relu', input_shape=(784,)))
    model.add(layers.Dropout({{uniform(0, 1)}}))
    model.add(layers.Dense({{choice([256, 128])}}, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(optimizer={{choice(['rmsprop', 'adam'])}}, 
                  loss='sparse_categorical_crossentropy',
                  metrics=['acc'])
    
    result = model.fit(train_x, train_y, 
                       batch_size={{choice([64, 128])}}, 
                       epochs=2,
                       verbose=2,
                       validation_split=0.2)
    #get the highest validation accuracy of the training epochs
    validation_acc = np.amax(result.history['val_acc'])
    validation_loss = np.amin(result.history['val_loss'])
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': validation_loss, 'status': STATUS_OK, 'model': model}

In [17]:
%%time
notebook_name = 'hyperas-test'

best_run, best_model = optim.minimize(model=mnist_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(),
                                      notebook_name=notebook_name)

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    from keras import layers, models
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dropout': hp.uniform('Dropout', 0, 1),
        'Dense': hp.choice('Dense', [256, 128]),
        'optimizer': hp.choice('optimizer', ['rmsprop', 'adam']),
        'batch_size': hp.choice('batch_size', [64, 128]),
    }

>>> Data
  1: 
  2: (train_x, train_y), (test_x, test_y) = mnist.load_data()
  3: train_x = train_x.astype('float32').reshape([60000, -1]) / 255.
  4: test_x = test_x.astype('float32').reshape([10000, -1]) / 255. 
  5: 
  6: 
  7: 
  8: 
>>> Resulting replaced keras model:

  1: def keras_fmin_fnct(space):
  2: 
  3:  

In [20]:
train_x, train_y, test_x, test_y = data()

best_model.evaluate(test_x, test_y)

10000/10000 [==============================] - 0s 26us/step


[0.09437107192506082, 0.9703]

In [21]:
best_run

{'Dense': 1, 'Dropout': 0.288662535902546, 'batch_size': 0, 'optimizer': 1}